LAST LINKs:


* https://colab.research.google.com/drive/1WIk2bxglElfZewOHboPFNj8H44_VAyKE?usp=sharing#scrollTo=Gw3IZYrfKl4Z
* https://medium.com/analytics-vidhya/fine-tune-a-roberta-encoder-decoder-model-trained-on-mlm-for-text-generation-23da5f3c1858
* https://huggingface.co/course/chapter7/7?fw=tf

LINKs:
* https://github.com/huggingface/notebooks/blob/main/examples/question_answering.ipynb

* https://github.com/Michael-M-Mike/Unibo-NLP-Assignments/blob/main/A2_Seq2Seq_Abstractive_Question_Answering_(QA)_on_CoQA/distilroberta_42.ipynb

# Assignment 2

**Credits**: Andrea Galassi, Federico Ruggeri, Paolo Torroni

**Keywords**: Transformers, Question Answering, CoQA

## Deadlines

* **December 11**, 2022: deadline for having assignments graded by January 11, 2023
* **January 11**, 2023: deadline for half-point speed bonus per assignment
* **After January 11**, 2023: assignments are still accepted, but there will be no speed bonus

## Overview

### Problem

Question Answering (QA) on [CoQA](https://stanfordnlp.github.io/coqa/) dataset: a conversational QA dataset.

### Task

Given a question $Q$, a text passage $P$, the task is to generate the answer $A$.<br>
$\rightarrow A$ can be: (i) a free-form text or (ii) unanswerable;

**Note**: an question $Q$ can refer to previous dialogue turns. <br>
$\rightarrow$ dialogue history $H$ may be a valuable input to provide the correct answer $A$.

### Models

We are going to experiment with transformer-based models to define the following models:

1.  $A = f_\theta(Q, P)$

2. $A = f_\theta(Q, P, H)$

where $f_\theta$ is the transformer-based model we have to define with $\theta$ parameters.

## The CoQA dataset

<center>
    <img src="https://drive.google.com/uc?export=view&id=16vrgyfoV42Z2AQX0QY7LHTfrgektEKKh" width="750"/>
</center>

For detailed information about the dataset, feel free to check the original [paper](https://arxiv.org/pdf/1808.07042.pdf).



## Rationales

Each QA pair is paired with a rationale $R$: it is a text span extracted from the given text passage $P$. <br>
$\rightarrow$ $R$ is not a requested output, but it can be used as an additional information at training time!

## Dataset Statistics

* **127k** QA pairs.
* **8k** conversations.
* **7** diverse domains: Children's Stories, Literature, Mid/High School Exams, News, Wikipedia, Reddit, Science.
* Average conversation length: **15 turns** (i.e., QA pairs).
* Almost **half** of CoQA questions refer back to **conversational history**.
* Only **train** and **validation** sets are available.

## Dataset snippet

The dataset is stored in JSON format. Each dialogue is represented as follows:

```
{
    "source": "mctest",
    "id": "3dr23u6we5exclen4th8uq9rb42tel",
    "filename": "mc160.test.41",
    "story": "Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. 
    Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. [...]" % <-- $P$
    "questions": [
        {
            "input_text": "What color was Cotton?",   % <-- $Q_1$
            "turn_id": 1
        },
        {
            "input_text": "Where did she live?",
            "turn_id": 2
        },
        [...]
    ],
    "answers": [
        {
            "span_start": 59,   % <-- $R_1$ start index
            "spand_end": 93,    % <-- $R_1$ end index
            "span_text": "a little white kitten named Cotton",   % <-- $R_1$
            "input_text" "white",   % <-- $A_1$      
            "turn_id": 1
        },
        [...]
    ]
}
```

### Simplifications

Each dialogue also contains an additional field ```additional_answers```. For simplicity, we **ignore** this field and only consider one groundtruth answer $A$ and text rationale $R$.

CoQA only contains 1.3% of unanswerable questions. For simplicity, we **ignore** those QA pairs.

# [0] Functions and imports

In [1]:
# %%capture
# %pip install datasets
# %pip install transformers
# %pip install tensorflow_addons
# %pip install allennlp-models
# %pip install plotly==5.13.1


# NOTE:
#     - SEED ED ERRORE
#     - LUNGHEZZA INPUTS E OUTPUTS - https://towardsdatascience.com/to-distil-or-not-to-distil-bert-roberta-and-xlnet-c777ad92f8
#     - GRANDEZZA DATASETS
#     - WARNINGS NELLA CREAZIONE DEL MODELLO E NEL TRAINING
#     - COME UTILIZZARE SPAN DI TESTO

In [2]:
from IPython.display import display_html, clear_output
from itertools import chain,cycle
from copy import deepcopy
import urllib.request
import transformers
import numpy as np
import json
import time
import os
import torch
import random 
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import GroupShuffleSplit
from datasets import *
from transformers import AutoTokenizer, PreTrainedTokenizerFast, EncoderDecoderModel
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, AdamW, DataCollatorForSeq2Seq

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # use the GPU
else:
    device = torch.device("cpu")  # use the CPU

print("Using device:", device)

Using device: cuda


In [4]:
import plotly.express as px

# Display dataframes
def display(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:left"><td style="vertical-align:top">'
        html_str+=f'<h4 style="text-align: left;">{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)
    
# Setting seeds for reproducibility
def set_reproducibility(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    transformers.set_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

    os.environ['TF_DETERMINISTIC_OPS'] = '1'

# Check tokenizer special tokens
def check_tokens(tokenizer):
    # Get the special tokens and their corresponding IDs
    special_tokens = tokenizer.special_tokens_map
    special_ids = tokenizer.convert_tokens_to_ids(list(special_tokens.values()))
    print("Special tokens:")
    for token_type, token_list in special_tokens.items():
        print(f"{token_type}: {token_list}")
    # Print the special tokens and their corresponding IDs
    for token, id in zip(special_tokens.keys(), special_ids):
        print(f"{token}: {id}")
        
# Compute metrics in the trainer
def compute_metrics(pred,tokenizer):
    labels = pred.label_ids
    preds = pred.predictions
    
    labels_text = tokenizer.batch_decode(labels, skip_special_tokens=True)
    preds_text = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    squad_scores=[]
    for i in range(len(preds_text)):
        squad_scores.append(compute_f1(str(preds_text[i]), str(labels_text[i])))
    mean_squad_f1 = sum(squad_scores)/len(squad_scores)

    return {"squad_f1_score": mean_squad_f1}

# # Generate Answers (on test set)
# def generate_answers(test_loader,model,tokenizer):
#     i=0
#     for batch in tqdm(test_loader):

#         example = batch['input_ids'].to(device)
#         att_mask = batch['attention_mask'].to(device)
#         generated_ids = model.generate(input_ids=example, 
#                                           attention_mask=att_mask,
#                                           max_length=tokenizer.model_max_length
#                                          )
#         ex = tokenizer.batch_decode(example, skip_special_tokens=True)
#         print(ex[0])
#         print(test_df['question'][i:i+8])

#         generated_answers = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
#         print(f'Generated ans: {generated_answers}')
#         true = batch["labels"]
#         ground_truth = tokenizer.batch_decode(true, skip_special_tokens=True)
#         print(f'True ans: {ground_truth}')
#         i+=8

### SQuAD metric

In [5]:
"""
Functions taken from [the official evaluation script]
(https://worksheets.codalab.org/rest/bundles/0x6b567e1cf2e041ec80d7098f031c5c9e/contents/blob/)
for SQuAD version 2.0.
"""
import collections
import re
import string
from typing import Callable, Sequence, TypeVar, Tuple


def make_qid_to_has_ans(dataset):
    qid_to_has_ans = {}
    for article in dataset:
        for p in article["paragraphs"]:
            for qa in p["qas"]:
                qid_to_has_ans[qa["id"]] = bool(qa["answers"])
    return qid_to_has_ans


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()


def compute_exact(a_pred: str, a_gold: str) -> int:
    return int(normalize_answer(a_pred) == normalize_answer(a_gold))


def compute_f1(a_pred: str, a_gold: str) -> float:
    pred_toks = get_tokens(a_pred)
    gold_toks = get_tokens(a_gold)
    common = collections.Counter(pred_toks) & collections.Counter(gold_toks)  # type: ignore[var-annotated]
    num_same = sum(common.values())
    if len(pred_toks) == 0 or len(gold_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return float(pred_toks == gold_toks)
    if num_same == 0:
        return 0.0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


_P = TypeVar("_P")
_G = TypeVar("_G")
_T = TypeVar("_T", int, float, Tuple[int, ...], Tuple[float, ...])


def metric_max_over_ground_truths(
    metric_fn: Callable[[_P, _G], _T], prediction: _P, ground_truths: Sequence[_G]
) -> _T:
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def get_metric_score(prediction: str, gold_answers: Sequence[str]) -> Tuple[int, float]:
    exact_scores = metric_max_over_ground_truths(compute_exact, prediction, gold_answers)
    f1_scores = metric_max_over_ground_truths(compute_f1, prediction, gold_answers)
    return exact_scores, f1_scores

## [Task 1] Remove unaswerable QA pairs

Write your own script to remove unaswerable QA pairs from both train and validation sets.

## Dataset Download


In [6]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)
        
def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def download_data(data_path, url_path, suffix):    
    if not os.path.exists(data_path):
        os.makedirs(data_path)
        
    data_path = os.path.join(data_path, f'{suffix}.json')

    if not os.path.exists(data_path):
        print(f"Downloading CoQA {suffix} data split... (it may take a while)")
        download_url(url=url_path, output_path=data_path)
        urllib.request.urlretrieve(url_path, filename=data_path)
        print("Download completed!")

In [7]:
# Train data
train_url = "https://nlp.stanford.edu/data/coqa/coqa-train-v1.0.json"
download_data(data_path='coqa', url_path=train_url, suffix='train')

# Test data
test_url = "https://nlp.stanford.edu/data/coqa/coqa-dev-v1.0.json"
download_data(data_path='coqa', url_path=test_url, suffix='test')  # <-- Why test? See next slides for an answer!

#### Data Inspection

Spend some time in checking accurately the dataset format and how to retrieve the tasks' inputs and outputs!

In [8]:
# Creating Dataframes and removing unanswerable questions
train_data = json.load((open('coqa/train.json')))
test_data = json.load((open('coqa/test.json')))

qas = pd.json_normalize(train_data['data'], ['questions'], ['source', 'id', 'story'])
ans = pd.json_normalize(train_data['data'], ['answers'],['id'])
train_val_df = pd.merge(qas,ans, left_on=['id','turn_id'], right_on=['id','turn_id'])
train_val_df = train_val_df.loc[train_val_df['input_text_y']!='unknown']

qas = pd.json_normalize(test_data['data'], ['questions'], ['source', 'id', 'story'])
ans = pd.json_normalize(test_data['data'], ['answers'],['id'])
test_df = pd.merge(qas,ans, left_on=['id','turn_id'], right_on=['id','turn_id'])
test_df = test_df.loc[test_df['input_text_y']!='unknown']

In [9]:
# Removing bad turns
train_val_df = train_val_df.loc[(train_val_df['bad_turn_x'] != 'True') & (train_val_df['bad_turn_y'] != 'True')]

# Removing equal text/answer entries
train_val_df = train_val_df[train_val_df.story != train_val_df.input_text_y]
test_df = test_df[test_df.story != test_df.input_text_y]

# Removing enties with empty answers
train_val_df = train_val_df[train_val_df['input_text_y'].str.len()>0]
test_df = test_df[test_df['input_text_y'].str.len()>0]

In [10]:
# Text preprocess
def preprocess(ds,columns):
#    ds = ds.replace(r'\n',' ', regex=True)
#    ds = ds.replace(r'[^\w\s]+', ' ', regex=True)
#     for feature in columns:
#         ds[feature] = ds[feature].str.lower().str.strip()
        
    return ds

columns = ['story', 'input_text_x', 'span_text', 'input_text_y']

train_val_df = preprocess(train_val_df,columns)
test_df = preprocess(test_df,columns)

## [Task 2] Train, Validation and Test splits

CoQA only provides a train and validation set since the test set is hidden for evaluation purposes.

We'll consider the provided validation set as a test set. <br>
$\rightarrow$ Write your own script to:
* Split the train data in train and validation splits (80% train and 20% val)
* Perform splits such that a dialogue appears in one split only! (i.e., split at dialogue level)
* Perform splitting using the following seed for reproducibility: 42

#### Reproducibility Memo

Check back tutorial 2 on how to fix a specific random seed for reproducibility!

In [11]:
# Train/Validation Split
set_reproducibility(42)

train_inds, val_inds = next(GroupShuffleSplit(test_size=.20, n_splits=2, random_state = 42).split(train_val_df, groups=train_val_df['id']))

train_df = train_val_df.iloc[train_inds]
val_df = train_val_df.iloc[val_inds].reset_index()

In [12]:
# Checking the Dataframes
print(f'Training set [{train_df.shape}]')
print(f'\tFeatures: {list(train_df.columns)}')
display(train_df.head())
#display(train_df.loc[11:15,['id', 'input_text_x', 'input_text_y', 'span_text']])

print(f'Validation set [{val_df.shape}]')
print(f'\tFeatures: {list(val_df.columns)}')
#display(val_df.head())
#display(val_df.loc[11:15,['id', 'input_text_x', 'input_text_y', 'span_text']])

print(f'Test set [{test_df.shape}]')
print(f'\tFeatures: {list(test_df.columns)}')
#display(test_df.head())
#display(test_df.loc[11:15,['id', 'input_text_x', 'input_text_y', 'span_text']])

Training set [(85823, 11)]
	Features: ['input_text_x', 'turn_id', 'bad_turn_x', 'source', 'id', 'story', 'span_start', 'span_end', 'span_text', 'input_text_y', 'bad_turn_y']


,input_text_x,turn_id,bad_turn_x,source,id,story,span_start,span_end,span_text,input_text_y,bad_turn_y
0,When was the Vat formally opened?,1,NaN,wikipedia,3zotghdk5ibi9cex97fepx7jetpso7,"The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made available online. \n\nThe Vatican Secret Archives were separated from the library at the beginning of the 17th century; they contain another 150,000 items. \n\nScholars have traditionally divided the history of the library into five periods, Pre-Lateran, Lateran, Avignon, Pre-Vatican and Vatican. \n\nThe Pre-Lateran period, comprising the initial days of the library, dated from the earliest days of the Church. Only a handful of volumes survive from this period, though some are very significant.",151,179,Formally established in 1475,It was formally established in 1475,NaN
1,what is the library for?,2,NaN,wikipedia,3zotghdk5ibi9cex97fepx7jetpso7,"The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made available online. \n\nThe Vatican Secret Archives were separated from the library at the beginning of the 17th century; they contain another 150,000 items. \n\nScholars have traditionally divided the history of the library into five periods, Pre-Lateran, Lateran, Avignon, Pre-Vatican and Vatican. \n\nThe Pre-Lateran period, comprising the initial days of the library, dated from the earliest days of the Church. Only a handful of volumes survive from this period, though some are very significant.",454,494,he Vatican Library is a research library,research,NaN
2,for what subjects?,3,NaN,wikipedia,3zotghdk5ibi9cex97fepx7jetpso7,"The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be re

Validation set [(21452, 12)]
	Features: ['index', 'input_text_x', 'turn_id', 'bad_turn_x', 'source', 'id', 'story', 'span_start', 'span_end', 'span_text', 'input_text_y', 'bad_turn_y']
Test set [(7917, 9)]
	Features: ['input_text_x', 'turn_id', 'source', 'id', 'story', 'span_start', 'span_end', 'span_text', 'input_text_y']


Now we check if there is any overlapping dialogue between train and validation set.

In [13]:
set_train = set(train_df['id'])
set_val = set(val_df['id'])

overlap = False
for i in set_train:
    if i in set_val:
        overlap = True
        break

print('Overlap' if overlap else 'No overlap')

No overlap


In [14]:
# Dataframes to Datasets
train_df_to_ds = train_df
val_df_to_ds = val_df
test_df_to_ds = test_df

train_df_to_ds = train_df_to_ds.rename(columns={'input_text_x': 'question', 'story': 'context',\
                                               'input_text_y': 'answer', 'span_text': 'text'})
val_df_to_ds = val_df_to_ds.rename(columns={'input_text_x': 'question', 'story': 'context',\
                                               'input_text_y': 'answer', 'span_text': 'text'})
test_df_to_ds = test_df_to_ds.rename(columns={'input_text_x': 'question', 'story': 'context',\
                                               'input_text_y': 'answer', 'span_text': 'text'})

Now, since the dataset is huge and we are more focused on the reasoning on our choices rather than obtaining the best results, we are going to extract a portion of it.
The next step is gonna be the truncation of the inputs lengths. The pre-trained models that are gonna be tested can process lengths up to 512, that is why our truncation will be at least equal to this value; moreover, we are going to sort the datasets according to the sum of the lengths of the 'context' and 'question' fields together, expecting to truncate the least possible number of examples.

In [15]:
# Combine 'context' and 'question' fields for each dataframe
train_df_to_ds['context_question'] = train_df_to_ds['context'] + train_df_to_ds['question']
val_df_to_ds['context_question'] = val_df_to_ds['context'] + val_df_to_ds['question']
test_df_to_ds['context_question'] = test_df_to_ds['context'] + test_df_to_ds['question']

# Define a function to compute the length of 'context_question'
def get_context_question_length(df):
    return (df['context_question'].apply(len))

# Compute the lengths of 'context_question' for each dataframe
train_lengths = get_context_question_length(train_df_to_ds)
val_lengths = get_context_question_length(val_df_to_ds)
test_lengths = get_context_question_length(test_df_to_ds)

# Sort each dataframe by length of 'context + question'
train_df_to_ds = train_df_to_ds.iloc[train_lengths.argsort()]
val_df_to_ds = val_df_to_ds.iloc[val_lengths.argsort()]
test_df_to_ds = test_df_to_ds.iloc[test_lengths.argsort()]

# Drop the 'context_question' column from each dataframe
train_df_to_ds = train_df_to_ds.drop('context_question', axis=1)
val_df_to_ds = val_df_to_ds.drop('context_question', axis=1)
test_df_to_ds = test_df_to_ds.drop('context_question', axis=1)

In [16]:
tokenizer=AutoTokenizer.from_pretrained("distilroberta-base")
assert isinstance(tokenizer, PreTrainedTokenizerFast)

In [17]:
def process_samples(sample):
    tokenized_data = tokenizer(sample["context"],sample["question"], truncation="only_first", max_length=512, padding="max_length")
    print(tokenized_data.keys())
    input_ids = tokenized_data["input_ids"]
    cls_index = input_ids.index(tokenizer.cls_token_id)

    if sample['context'][0]==-1:
        start_position = cls_index
        end_position = cls_index
    else:
        #gold_text = sample['text']
        gold_text = sample['text'][sample['span_start']:sample['span_end']]
        start_char = sample['span_start']
        end_char = sample['span_end']
        print(sample['context'][start_char:end_char])

        if sample['context'][start_char-1:end_char-1] == gold_text:
            start_char -= -1
            end_char -= 1
        elif sample['context'][start_char-2:end_char-2] == gold_text:
            start_char -= 2
            end_char -= 2

        start_token = tokenized_data.char_to_token(start_char)
        end_token = tokenized_data.char_to_token(end_char)

        if start_token is None:
            start_token = 512
        if end_token is None:
            end_token = 512

        start_position = start_token
        end_position = end_token
        print(f"Golden Text: {sample['text']}")
        print(print(sample['context'][start_char:end_char]))
    return {'input_ids': tokenized_data['input_ids'],
            'attention_mask': tokenized_data['attention_mask'],
            'start_positions':start_position,
            'end_positions':end_position}
    

In [18]:
from datasets import Dataset
train_ds = Dataset.from_pandas(train_df_to_ds)
eval_ds = Dataset.from_pandas(val_df_to_ds)
test_ds = Dataset.from_pandas(test_df_to_ds)
flat_train = train_ds.flatten()
flat_val = eval_ds.flatten()
flat_test = test_ds.flatten()
#print(flat_train[0])
processed_train_data =flat_train.select(range(3000)).map(process_samples)
processed_val_data = flat_val.select(range(3000)).map(process_samples)
processed_test_data = flat_test.select(range(1000)).map(process_samples)


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask'])
She has a hamster
Golden Text: She has a hamster
She has a hamster
None
dict_keys(['input_ids', 'attention_mask'])
Her bunny's name is Snowball.
Golden Text: Her bunny's name is Snowball.
Her bunny's name is Snowball.
None
dict_keys(['input_ids', 'attention_mask'])
Polly plays in her cage
Golden Text: Polly plays in her cage
Polly plays in her cage
None
dict_keys(['input_ids', 'attention_mask'])
Her cat's name is Kitty
Golden Text: Her cat's name is Kitty
Her cat's name is Kitty
None
dict_keys(['input_ids', 'attention_mask'])
Molly likes animals
Golden Text: Molly likes animals
Molly likes animals
None
dict_keys(['input_ids', 'attention_mask'])
Her bunny's name is Snowball.
Golden Text: Her bunny's name is Snowball.
Her bunny's name is Snowball.
None
dict_keys(['input_ids', 'attention_mask'])
Her hamster's name is Barry
Golden Text: Her hamster's name is Barry
Her hamster's name is Barry
None
dict_keys(['input_ids', 'attention_mask'])
She has 

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask'])
the puppy
Golden Text: the puppy
the puppy
None
dict_keys(['input_ids', 'attention_mask'])
nap.
Golden Text: nap.
nap.
None
dict_keys(['input_ids', 'attention_mask'])
Mary was happy when Poppy showed her all that love
Golden Text: Mary was happy when Poppy showed her all that love
Mary was happy when Poppy showed her all that love
None
dict_keys(['input_ids', 'attention_mask'])
outside
Golden Text: outside
outside
None
dict_keys(['input_ids', 'attention_mask'])
Mary wanted to play with him
Golden Text: Mary wanted to play with him
Mary wanted to play with him
None
dict_keys(['input_ids', 'attention_mask'])
love
Golden Text: love
love
None
dict_keys(['input_ids', 'attention_mask'])
When Poppy was tired of that
Golden Text: When Poppy was tired of that
When Poppy was tired of that
None
dict_keys(['input_ids', 'attention_mask'])
One day
Golden Text: One day
One day
None
dict_keys(['input_ids', 'attention_mask'])
eat
Golden Text: eat
eat
None
dict

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'attention_mask'])
But her baby sister is crying so loud that Jenny can't hear herself, so she was angry!
Golden Text: But her baby sister is crying so loud that Jenny can't hear herself, so she was angry!
But her baby sister is crying so loud that Jenny can't hear herself, so she was angry!
None
dict_keys(['input_ids', 'attention_mask'])
 He was afraid of being chased by a squirre
Golden Text:  He was afraid of being chased by a squirre
 He was afraid of being chased by a squirre
None
dict_keys(['input_ids', 'attention_mask'])
asked if Kim wanted to play. 
Golden Text: asked if Kim wanted to play. 
asked if Kim wanted to play. 
None
dict_keys(['input_ids', 'attention_mask'])
Jenny loves singing
Golden Text: Jenny loves singing
Jenny loves singing
None
dict_keys(['input_ids', 'attention_mask'])
she was angry
Golden Text: she was angry
she was angry
None
dict_keys(['input_ids', 'attention_mask'])
but John thought she was a good friend.
Golden Text: but John thoug

In [19]:
from transformers import AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained("distilroberta-base")

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be 

In [20]:
columns_to_return = ['input_ids','attention_mask', 'start_positions', 'end_positions']
processed_train_data.set_format(type='pt', columns=columns_to_return)
processed_val_data.set_format(type='pt', columns=columns_to_return)
processed_test_data.set_format(type='pt',columns=columns_to_return)

In [21]:
from sklearn.metrics import f1_score

def compute_f1_metrics(pred):    
    start_labels = pred.label_ids[0]
    start_preds = pred.predictions[0].argmax(-1)
    end_labels = pred.label_ids[1]
    end_preds = pred.predictions[1].argmax(-1)
    
    f1_start = f1_score(start_labels, start_preds, average='macro')
    f1_end = f1_score(end_labels, end_preds, average='macro')
    
    return {
        'f1_start': f1_start,
        'f1_end': f1_end,
    }

In [22]:
torch.cuda.empty_cache()

# Training the model may take around 15 minutes.
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='model_results5',          # output directory
    overwrite_output_dir=True,
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,    #batch size for evaluation
    warmup_steps=20,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=None,            # directory for storing logs
    logging_steps=50
)

trainer = Trainer(
    model=model, # the instantiated 🤗 Transformers model to be trained
    args=training_args, # training arguments, defined above
    train_dataset=processed_train_data, # training dataset
    eval_dataset=processed_val_data,  #evaluation dataset
    compute_metrics=compute_f1_metrics,
)

os.environ["WANDB_DISABLED"]="true"
trainer.train()

d:\envs\tf\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


  0%|          | 0/750 [00:00<?, ?it/s]

{'loss': 4.8981, 'learning_rate': 4.794520547945205e-05, 'epoch': 0.13}
{'loss': 4.1699, 'learning_rate': 4.452054794520548e-05, 'epoch': 0.27}
{'loss': 3.9367, 'learning_rate': 4.1095890410958905e-05, 'epoch': 0.4}
{'loss': 3.7575, 'learning_rate': 3.767123287671233e-05, 'epoch': 0.53}
{'loss': 3.4745, 'learning_rate': 3.424657534246575e-05, 'epoch': 0.67}
{'loss': 3.4697, 'learning_rate': 3.082191780821918e-05, 'epoch': 0.8}
{'loss': 3.0908, 'learning_rate': 2.7397260273972603e-05, 'epoch': 0.93}
{'loss': 2.9488, 'learning_rate': 2.3972602739726026e-05, 'epoch': 1.07}
{'loss': 2.7219, 'learning_rate': 2.0547945205479453e-05, 'epoch': 1.2}
{'loss': 2.6615, 'learning_rate': 1.7123287671232875e-05, 'epoch': 1.33}
{'loss': 2.8364, 'learning_rate': 1.3698630136986302e-05, 'epoch': 1.47}
{'loss': 2.5523, 'learning_rate': 1.0273972602739726e-05, 'epoch': 1.6}
{'loss': 2.4574, 'learning_rate': 6.849315068493151e-06, 'epoch': 1.73}
{'loss': 2.6487, 'learning_rate': 3.4246575342465754e-06, 'ep

TrainOutput(global_step=750, training_loss=3.206327952067057, metrics={'train_runtime': 283.4395, 'train_samples_per_second': 21.169, 'train_steps_per_second': 2.646, 'train_loss': 3.206327952067057, 'epoch': 2.0})

In [23]:
# The evaluation may take around 30 seconds
trainer.evaluate(processed_test_data)

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 2.912344455718994,
 'eval_f1_start': 0.21023110924401583,
 'eval_f1_end': 0.2180315787322114,
 'eval_runtime': 16.592,
 'eval_samples_per_second': 60.27,
 'eval_steps_per_second': 7.534,
 'epoch': 2.0}

In [34]:
def generate_answers(test_loader, model, tokenizer):
    for batch in tqdm(test_loader):
        i=0
        for question in batch['question']:
            inputs=tokenizer.encode_plus(question,batch['context'][0],return_tensors='pt',max_length=514,truncation='only_second')
            input_ids = inputs['input_ids'].tolist()[0]
            inputs.to('cuda')

            text_tokens=tokenizer.convert_ids_to_tokens(input_ids)
            answer_model = model(**inputs)

            answer_start = torch.argmax(
                answer_model['start_logits']
            )
            answer_end = torch.argmax(answer_model['end_logits']) + 1  # Get the most likely end of answer with the argmax of the score

            answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

            print(f"Question: {question}")
            print(f"Answer: {answer}\n")
            print(f"True Answer: {batch['answer'][i]}\n")
            i+=1


In [35]:
torch.cuda.empty_cache()
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=8)

# Generate answers
generate_answers(test_loader=test_loader,tokenizer=tokenizer,model=model)

  0%|          | 1/990 [00:00<04:06,  4.02it/s]

Question: Why?
Answer: 

True Answer: Because her baby sister is crying so loud that Jenny can't hear herself.

Question: Why?
Answer: 

True Answer: He was afraid of being chased by a squirre

Question: Why?
Answer: 

True Answer: She asked her if she wanted to play

Question: Who loves singing?
Answer: 

True Answer: Jenny.

Question: How is she feeling?
Answer:  Jenny was getting tired,

True Answer: Angry.

Question: Did he?
Answer:  Jenny was getting tired, but she had an idea. She would give her baby sister one of her toy horses. The baby loved horses! The baby played with the horse, but then she started crying again.

True Answer: yes

Question: of what?
Answer: 

True Answer: a black cat

Question: For what?
Answer: 

True Answer: to buy presents

Question: What else?
Answer: 

True Answer: a bowl

Question: Does Jenny have a sister?
Answer: 

True Answer: Yes.

Question: anything else?
Answer: 

True Answer: a blue feather

Question: Is Jenny older or younger?
Answer:  Whisker

  0%|          | 2/990 [00:00<02:56,  5.59it/s]

Question: Who is Whiskers?
Answer: 

True Answer: a cat

Question: What did he say?
Answer: 

True Answer: no

Question: whats on the bowl?
Answer: 

True Answer: a picture

Question: What did she say?
Answer: 

True Answer: yes

Question: Why is Lisa excited?
Answer: Lisa is excited because

True Answer: Saturday, Whiskers turns two

Question: How did she ultimately solve that?
Answer: 

True Answer: She gave the baby a pacifier.

Question: What did she try just before that?
Answer: 

True Answer: She gave her a toy horse.

Question: When did they go home?
Answer: 

True Answer: Dinnertime.

Question: What does it look like?
Answer: 

True Answer: black with a white spot

Question: Does it match Whiskers?
Answer: 

True Answer: definitely



  0%|          | 4/990 [00:00<02:29,  6.60it/s]

...

Question: How did he explain the noise?
Answer: 

True Answer: Guy's cleaning out the fourth form.

Question: what is the population?
Answer: 

True Answer: 2.06 million.

Question: How old was Kody when he was diagnosed with diabetes?
Answer: Dave Duerson had classic pathology of CTE and no evidence of any other disease,"

True Answer: six

Question: Is Taken a slow relaxed read?
Answer: Dave Duerson had classic pathology of CTE and no evidence of any other disease,"

True Answer: No

Question: Who was shooting on the lawn when Mr and Miss Faulkner rode down from High Down?
Answer: 

True Answer: Marian, Caroline, and Clara

Question: What?
Answer: Your mother just can't be bringing me meals like this all the time,"

True Answer: a paper carrier bag

Question: What prestigious list did it make that year?
Answer: Your mother just can't be bringing me meals like this all the time,"

True Answer: The New York Times' list of Best Books

Question: With who?
Answer: Your mother just can't b

 90%|████████▉ | 887/990 [01:55<00:17,  5.91it/s]

Question: What is the ranking in the country in terms of people studying?
Answer: Your mother just can't be bringing me meals like this all the time,"

True Answer: the fourth largest population

Question: what's the official name?
Answer: Your mother just can't be bringing me meals like this all the time,"

True Answer: the Republic of Slovenia

Question: What areas of Davis' brain were impacted?
Answer: Your mother just can't be bringing me meals like this all the time,"

True Answer: areas that control judgment, inhibition, impulse control, mood and memory



 90%|████████▉ | 887/990 [01:55<00:13,  7.68it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
